In [1]:
import numpy as np

TRAIN_SIZE, VAL_SIZE = 40000, 10000
dataset_prefix = "cifar100" 
num_classes = 100

train_data = np.load("./hw5_data/{}_train.npy".format(dataset_prefix))
test_data = np.load("./hw5_data/{}_test.npy".format(dataset_prefix))

# Note: I am unable to use sklearn at the moment due to package inconsistencies
# Split train data to train/val/test
train_images = train_data[:TRAIN_SIZE, :]
val_images = train_data[TRAIN_SIZE:, :]

test_images = test_data.reshape(-1, 3, 32, 32)

In [ ]:
test_data.shape

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

def imshow(image, title=None):
    image = Image.fromarray((np.moveaxis(image, (0,1,2), (2, 0, 1))*255).astype("uint8"), 'RGB')
    fig, ax = plt.subplots(1, figsize=(2,2))
    ax.imshow(image)
    if title is not None:
        plt.title(title)
for i in range(10):
    imshow(train_images[i], title='Image')

In [65]:
# PyTorch: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

import torch
import torch.nn as nn
import torch.nn.functional as F

# CNN Architecture: (Conv2d->ReLU->Maxpool) x M -> (Linear->ReLU) x N -> Softmax
# Maxpool only used at most thrice to prevent erosion of signals (HxW will keep shrinking for each pool)
class Net(nn.Module):
    def __init__(self, S=1, P=0, convlist=[(10, 5), (10, 5)], numpools=2, pool=2, pool_S=2, fclist=[20, 10], drop=0.5, batch=1):
        super(Net, self).__init__()
        
        # Conv Filters list; First in_channel is 3 (RGB image)
        self.numconv = len(convlist)
        self.conv1 = nn.Conv2d(3, convlist[0][0], convlist[0][1], stride=S, padding=P)
        self.conv2 = nn.Conv2d(convlist[0][0], convlist[1][0], convlist[1][1], stride=S, padding=P)
        if self.numconv > 2:
            self.conv3 = nn.Conv2d(convlist[1][0], convlist[2][0], convlist[2][1], stride=S, padding=P)
            if self.numconv > 3:
                self.conv4 = nn.Conv2d(convlist[2][0], convlist[3][0], convlist[3][1], stride=S, padding=P)
                if self.numconv > 4:
                    self.conv5 = nn.Conv2d(convlist[3][0], convlist[4][0], convlist[4][1], stride=S, padding=P)
        
        # Maxpool
        self.numpools = numpools
        self.pool = nn.MaxPool2d(pool, stride=pool_S)
               
        # FC Layers list and batchnorms
#         print(convlist, numpools)
        self.l = int(convlist[-1][0] * (32 / (2**numpools))**2)
        self.numfc = len(fclist)+1
        self.fc1 = nn.Linear(self.l, fclist[0], bias=True)
        self.bn1 = nn.BatchNorm1d(num_features=fclist[0])
        
        if self.numfc > 2:
            self.fc2 = nn.Linear(fclist[0], fclist[1], bias=True)
            self.bn2 = nn.BatchNorm1d(num_features=fclist[1])
            if self.numfc > 3:
                self.fc3 = nn.Linear(fclist[1], fclist[2], bias=True)
                self.bn3 = nn.BatchNorm1d(num_features=fclist[2])
                if self.numfc > 4:
                    self.fc4 = nn.Linear(fclist[2], fclist[3], bias=True)
                    
        self.fcf = nn.Linear(fclist[-1], 100, bias=True)
        
        # Softmax
        self.softmax = nn.Softmax(dim=1)
        
        # Dropout and batch size 
        self.dropout = nn.Dropout(drop)
        self.batch = batch
        print("Batch size: {:}".format(self.batch))
        

    def forward(self, x):
        # Convolution filters
        x = self.pool(F.relu(self.dropout(self.conv1(x))))
        x = F.relu(self.dropout(self.conv2(x)))
        if self.numpools > 1:
            x = self.pool(x)
        if self.numconv > 2:
            x = F.relu(self.dropout(self.conv3(x)))
            if self.numpools > 2:
                x = self.pool(x)
            if self.numconv > 3:
                x = F.relu(self.dropout(self.conv4(x)))
                if self.numpools > 3:
                    x = self.pool(x)
                if self.numconv > 4:
                    x = F.relu(self.dropout(self.conv5(x)))
                    if self.numpools > 4:
                        x = self.pool(x)
                        
        x = x.view(-1, self.l)
        
        # FC layers
        x = F.relu(self.bn1(self.fc1(x)))
        if self.numfc > 2:
            x = F.relu(self.bn2(self.fc2(x)))
            if self.numfc > 3:
                x = F.relu(self.bn3(self.fc3(x)))
                if self.numfc > 4:
                    x = F.relu(self.fc4(x))
                    
        x = F.relu(self.fcf(x))
        x = self.softmax(x)
        
        return x



In [66]:
import pandas as pd
import torch.optim as optim

def train(S, P, convlist, numpools, pool, pool_S, fclist, drop, opt, lr, mom, batch):
    # NN INITIALIZATION
    net = Net(S=S, P=P, convlist=convlist, numpools=numpools, pool=pool, pool_S=pool_S, fclist=fclist, drop=drop, batch=batch)    
    
    criterion = nn.CrossEntropyLoss()
    # Using Adam Optimizer
    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.9,0.999))
    
    trainloader = torch.utils.data.DataLoader(train_images, batch_size=batch,
                                              shuffle=True, num_workers=2)
    epsilon = 0.0
    losses = []
    accs = []
    vals = []
    for epoch in range(100):  # loop over the dataset multiple times 
        # update lr after 30 epochs
        if epoch == 30:
            optimizer = optim.Adam(net.parameters(), lr=lr*.1, betas=(0.9,0.999))
            
        running_loss = 0.0
        train_correct, train_total = 0, 0
        for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[:, :-1]
            labels = data[:, -1]
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            if inputs.shape[0] == batch:
                outputs = net(inputs.reshape(batch, 3, 32, 32).float())
                outputs = outputs.reshape(batch, -1)
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels.long())
                loss.backward()
                optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % (40000//(10*batch)) == ((40000//(10*batch))-1):    # print every 1/10 epoch
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / (40000//(10*batch))))
                print('Accuracy of the network on train images: %d %%' % (
                    100 * train_correct / train_total))
                
                losses.append(running_loss/(40000//(10*batch)))
                accs.append(100 * train_correct / train_total)
                running_loss = 0.0
                
                # val accuracy
                valloader = torch.utils.data.DataLoader(val_images, batch_size=VAL_SIZE,
                                                          shuffle=True, num_workers=2)
                correct = 0
                total = 0
                with torch.no_grad():
                    
                    for data in valloader:
                        images, labels = data[:, :-1], data[:, -1]
                        outputs = net(images.reshape(VAL_SIZE, 3, 32, 32).float())
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                print('Accuracy of the network on the 10000 validation images: %d %%' % (
                    100 * correct / total))
                vals.append(100 * correct / total)
                

        if (vals[-1] - vals[-9] < (epsilon-1)):
            break

    print('Finished Training')
    return net, losses, vals

In [67]:
# Perform random search to tune hyperparameters; more efficient than grid search since it allows for identifying important
# parameters more quickly, and avoids prioritizing less important ones (Stanford CS231)

def random_search(num_iter=10):
    # S = 1 as good practice and simplified dimensional constraints
    # pool = 2, pool_S=2 as convention according to cs231
    kernel = (2*np.random.randint(0, 2, size=num_iter)) + 3
    pad = kernel // 2 # If conv kernel=5, pad = 2; if conv kernel=3, pad = 1; this simplifies dimensional constraints
    
    convlist = 2**np.random.randint(5, 8, size=(num_iter, 5)) # convlist.size in [3, 3], conv in [16, 128]
    numpools = np.random.randint(2, 3, size=num_iter) # numpools in [1, 3]
    
    fclist = 2**np.random.randint(6, 8, size=(num_iter, 5)) # fclist.size in [2, 3], fc in [16, 128]
    opt = 'adam'
    
    lr = np.random.randint(4, 5, size=num_iter) # lr in {0.001, 0.0001}
    mom = np.random.uniform(0.8, 0.99, size=num_iter) # mom in [0.7, 0.99]
    drop = np.random.uniform(0.4, 0.81, size=num_iter) # dropout in [0.4, 0.8]
    batch = 2**(np.random.randint(3, 7, size=num_iter)) # batch in [8, 64]
    #convlist=[(convlist[i][j], kernel[i]) for j in range(np.random.randint(3,4))]
    #fclist=fclist[i][:np.random.randint(2,3)]
    for i in range(num_iter):
        net, losses, vals = train(S=1, P=pad[i], convlist=[(32, kernel[i]), (64, kernel[i]), (128, kernel[i])],
                                  numpools=numpools[i], pool=2, pool_S=2, fclist=[128,512], 
                                  drop=drop[i], opt=opt, lr=0.1**lr[i], mom=mom[i], batch=int(batch[i]))
        
        print("Trained NN{:}".format(i+10))
        
        # Save NN
        PATH = 'cifar/cifar_net{:}.pth'.format(i+10)
        torch.save(net.state_dict(), PATH)
        
        # Save losses
        pd.DataFrame(losses).to_csv("cifar/cifarlosses{:}.csv".format(i+10))
        pd.DataFrame(vals).to_csv("cifar/cifarvals{:}.csv".format(i+10))
        # Save parameters
        with open('cifar/cifarparams{:}.txt'.format(i+10), 'w+') as f:
            f.write("{:}\n lr: {:}\n mom: {:}\n drop: {:}\n batch: {:}\n numpools: {:}".format(net.parameters, 0.1**lr[i], 
                                                                                               mom[i], drop[i], batch[i], 
                                                                                               numpools[i]))
        
        print("Saved NN{:}".format(i+10))


In [68]:
random_search(1)

Batch size: 32
[1,   125] loss: 4.605
Accuracy of the network on train images: 1 %
Accuracy of the network on the 10000 validation images: 1 %
[1,   250] loss: 4.603
Accuracy of the network on train images: 2 %
Accuracy of the network on the 10000 validation images: 3 %
[1,   375] loss: 4.590
Accuracy of the network on train images: 2 %
Accuracy of the network on the 10000 validation images: 3 %
[1,   500] loss: 4.593
Accuracy of the network on train images: 2 %
Accuracy of the network on the 10000 validation images: 3 %
[1,   625] loss: 4.586
Accuracy of the network on train images: 2 %
Accuracy of the network on the 10000 validation images: 3 %
[1,   750] loss: 4.585
Accuracy of the network on train images: 2 %
Accuracy of the network on the 10000 validation images: 4 %
[1,   875] loss: 4.579
Accuracy of the network on train images: 3 %
Accuracy of the network on the 10000 validation images: 5 %
[1,  1000] loss: 4.575
Accuracy of the network on train images: 3 %
Accuracy of the netwo

Accuracy of the network on the 10000 validation images: 13 %
[7,   625] loss: 4.493
Accuracy of the network on train images: 13 %
Accuracy of the network on the 10000 validation images: 13 %
[7,   750] loss: 4.483
Accuracy of the network on train images: 13 %
Accuracy of the network on the 10000 validation images: 13 %
[7,   875] loss: 4.485
Accuracy of the network on train images: 13 %
Accuracy of the network on the 10000 validation images: 13 %
[7,  1000] loss: 4.487
Accuracy of the network on train images: 13 %
Accuracy of the network on the 10000 validation images: 13 %
[7,  1125] loss: 4.482
Accuracy of the network on train images: 14 %
Accuracy of the network on the 10000 validation images: 13 %
[7,  1250] loss: 4.487
Accuracy of the network on train images: 14 %
Accuracy of the network on the 10000 validation images: 13 %
[8,   125] loss: 4.475
Accuracy of the network on train images: 15 %
Accuracy of the network on the 10000 validation images: 13 %
[8,   250] loss: 4.484
Accura

Accuracy of the network on the 10000 validation images: 17 %
[13,  1000] loss: 4.445
Accuracy of the network on train images: 18 %
Accuracy of the network on the 10000 validation images: 16 %
[13,  1125] loss: 4.434
Accuracy of the network on train images: 18 %
Accuracy of the network on the 10000 validation images: 16 %
[13,  1250] loss: 4.451
Accuracy of the network on train images: 18 %
Accuracy of the network on the 10000 validation images: 16 %
[14,   125] loss: 4.447
Accuracy of the network on train images: 18 %
Accuracy of the network on the 10000 validation images: 16 %
[14,   250] loss: 4.440
Accuracy of the network on train images: 18 %
Accuracy of the network on the 10000 validation images: 17 %
[14,   375] loss: 4.437
Accuracy of the network on train images: 18 %
Accuracy of the network on the 10000 validation images: 16 %
[14,   500] loss: 4.432
Accuracy of the network on train images: 19 %
Accuracy of the network on the 10000 validation images: 16 %
[14,   625] loss: 4.44

Accuracy of the network on the 10000 validation images: 18 %
[20,   125] loss: 4.411
Accuracy of the network on train images: 21 %
Accuracy of the network on the 10000 validation images: 18 %
[20,   250] loss: 4.412
Accuracy of the network on train images: 21 %
Accuracy of the network on the 10000 validation images: 18 %
[20,   375] loss: 4.408
Accuracy of the network on train images: 21 %
Accuracy of the network on the 10000 validation images: 18 %
[20,   500] loss: 4.402
Accuracy of the network on train images: 22 %
Accuracy of the network on the 10000 validation images: 19 %
[20,   625] loss: 4.414
Accuracy of the network on train images: 22 %
Accuracy of the network on the 10000 validation images: 18 %
[20,   750] loss: 4.398
Accuracy of the network on train images: 22 %
Accuracy of the network on the 10000 validation images: 18 %
[20,   875] loss: 4.411
Accuracy of the network on train images: 22 %
Accuracy of the network on the 10000 validation images: 19 %
[20,  1000] loss: 4.41

Accuracy of the network on the 10000 validation images: 20 %
[26,   500] loss: 4.394
Accuracy of the network on train images: 23 %
Accuracy of the network on the 10000 validation images: 19 %
[26,   625] loss: 4.389
Accuracy of the network on train images: 24 %
Accuracy of the network on the 10000 validation images: 19 %
[26,   750] loss: 4.386
Accuracy of the network on train images: 24 %
Accuracy of the network on the 10000 validation images: 19 %
[26,   875] loss: 4.388
Accuracy of the network on train images: 24 %
Accuracy of the network on the 10000 validation images: 20 %
[26,  1000] loss: 4.395
Accuracy of the network on train images: 24 %
Accuracy of the network on the 10000 validation images: 20 %
[26,  1125] loss: 4.386
Accuracy of the network on train images: 24 %
Accuracy of the network on the 10000 validation images: 20 %
[26,  1250] loss: 4.393
Accuracy of the network on train images: 23 %
Accuracy of the network on the 10000 validation images: 20 %
[27,   125] loss: 4.38

...

Accuracy of the network on the 10000 validation images: 21 %
[70,   625] loss: 4.349
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 22 %
[70,   750] loss: 4.345
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[70,   875] loss: 4.350
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[70,  1000] loss: 4.353
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[70,  1125] loss: 4.346
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[70,  1250] loss: 4.341
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[71,   125] loss: 4.336
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[71,   250] loss: 4.34

Accuracy of the network on the 10000 validation images: 21 %
[76,  1000] loss: 4.351
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[76,  1125] loss: 4.342
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[76,  1250] loss: 4.345
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 22 %
[77,   125] loss: 4.342
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[77,   250] loss: 4.341
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 22 %
[77,   375] loss: 4.353
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[77,   500] loss: 4.339
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[77,   625] loss: 4.36

Accuracy of the network on the 10000 validation images: 22 %
[83,   125] loss: 4.343
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[83,   250] loss: 4.342
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 22 %
[83,   375] loss: 4.340
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[83,   500] loss: 4.329
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[83,   625] loss: 4.345
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[83,   750] loss: 4.348
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[83,   875] loss: 4.344
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[83,  1000] loss: 4.34

Accuracy of the network on the 10000 validation images: 21 %
[89,   500] loss: 4.349
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[89,   625] loss: 4.340
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[89,   750] loss: 4.341
Accuracy of the network on train images: 28 %
Accuracy of the network on the 10000 validation images: 21 %
[89,   875] loss: 4.321
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[89,  1000] loss: 4.343
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[89,  1125] loss: 4.349
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 22 %
[89,  1250] loss: 4.335
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[90,   125] loss: 4.33

Accuracy of the network on the 10000 validation images: 22 %
[95,   875] loss: 4.335
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[95,  1000] loss: 4.335
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 22 %
[95,  1125] loss: 4.341
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 22 %
[95,  1250] loss: 4.331
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 21 %
[96,   125] loss: 4.335
Accuracy of the network on train images: 29 %
Accuracy of the network on the 10000 validation images: 22 %
[96,   250] loss: 4.322
Accuracy of the network on train images: 30 %
Accuracy of the network on the 10000 validation images: 22 %
[96,   375] loss: 4.323
Accuracy of the network on train images: 30 %
Accuracy of the network on the 10000 validation images: 22 %
[96,   500] loss: 4.33

In [69]:
# Load NN
PATH = 'cifar/cifar_net{:}.pth'.format(10)
BATCH = 100

net = Net(S=1, P=1, convlist=[(32, 3), (64, 3), (128, 3)], numpools=2, pool=2, pool_S=2, fclist=[128, 512], 
          drop=0.6244900211791328, batch=BATCH)
net.load_state_dict(torch.load(PATH))

# predictions on test set
testloader = torch.utils.data.DataLoader(test_images, batch_size=BATCH, shuffle=False, num_workers=2)

predictions = []
with torch.no_grad():
    for i, data in enumerate(testloader):
        outputs = net(data.reshape(BATCH, 3, 32, 32).float())
        predictions.extend(torch.max(outputs.data, 1)[1])
        print(i)
        

pd.DataFrame(predictions).to_csv("predictions10.csv")

Batch size: 100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
